In [32]:
import pandas as pd
import numpy as np

import win32com.client as win32
from datetime import datetime
import pandas as pd
import numpy as np
import math
import os

In [33]:
# # To Import Files from Jupyterhub/Lab

# two_month= pd.read_csv('twomonthback.csv')
# previous= pd.read_csv('previouscsv')
# current= pd.read_csv('current.csv')

# indefault = pd.read_excel("RDB_Dump_(Month-wise) (January 2022).xlsx", sheet_name="In Default")
# outdefault= pd.read_excel("RDB_Dump_(Month-wise) (January 2022).xlsx", sheet_name="Out of Default")
# mapping_table= pd.read_excel('mapping_table.xlsx')

In [34]:
# Importing Files From Local 

two_month= pd.read_csv("C:/Users/admin/Desktop/Sagar/Final_Test_Run/twomonthback.csv")
previous= pd.read_csv("C:/Users/admin/Desktop/Sagar/Final_Test_Run//previous.csv")
current= pd.read_csv("C:/Users/admin/Desktop/Sagar/Final_Test_Run/current.csv")

indefault = pd.read_excel("C:/Users/admin/Desktop/Sagar/Final_Test_Run/RDB_Dump.xlsx", sheet_name="In Default")
outdefault= pd.read_excel("C:/Users/admin/Desktop/Sagar/Final_Test_Run/RDB_Dump.xlsx", sheet_name="Out of Default")
mapping_table= pd.read_excel("C:/Users/admin/Desktop/Sagar/Final_Test_Run/External Rating Map.xlsx")

# pdf = read_pdf('Errors Description - New v2 - March 08 2023.pdf')

In [35]:
# Bank Details for Automated Email Process
Bank_data=pd.read_excel("C:/Users/admin/Desktop/Sagar/Final_Test_Run/CRILC - Contact Details - For ReBIT - March 08 2023.xlsx")

In [36]:
current.columns= ['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN', 'CIN', 'LEI',
       'Borrower_Customer_Name', 'Assets_Classification',
       'Date_of_NPA_Classification', 'Group_Name', 'Industry_Code','Industry_Name',
       'Amount_technically_prudentially', 'External_Rating',
       'Borrower_Classified_as_Fraud', 'Special_Mention_Accounts',
       'Date_of_SMA_Classification', 'Funded_Amount_Outstanding_Total',
       'Non_Funded_Amount_Outstanding_Total', 'Total_Credit_Exposure',
       'Bank_Name', 'Bank_Group_Level3', 'Lead_Bank_Name',
       'Total_Investment_Exposure', 'Total_Exposure']

In [37]:
previous.columns =['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN', 'CIN', 'LEI',
       'Borrower_Customer_Name', 'Assets_Classification',
       'Date_of_NPA_Classification', 'Group_Name', 'Industry_Code','Industry_Name',
       'Amount_technically_prudentially', 'External_Rating',
       'Borrower_Classified_as_Fraud', 'Special_Mention_Accounts',
       'Date_of_SMA_Classification', 'Funded_Amount_Outstanding_Total',
       'Non_Funded_Amount_Outstanding_Total', 'Total_Credit_Exposure',
       'Bank_Name', 'Bank_Group_Level3', 'Lead_Bank_Name',
       'Total_Investment_Exposure', 'Total_Exposure']

In [38]:
two_month.columns = ['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN', 'CIN', 'LEI',
       'Borrower_Customer_Name', 'Assets_Classification',
       'Date_of_NPA_Classification', 'Group_Name', 'Industry_Code','Industry_Name',
       'Amount_technically_prudentially', 'External_Rating',
       'Borrower_Classified_as_Fraud', 'Special_Mention_Accounts',
       'Date_of_SMA_Classification', 'Funded_Amount_Outstanding_Total',
       'Non_Funded_Amount_Outstanding_Total', 'Total_Credit_Exposure',
       'Bank_Name', 'Bank_Group_Level3', 'Lead_Bank_Name',
       'Total_Investment_Exposure', 'Total_Exposure']

In [39]:
#Temp_Borrower
Temp_Borrower = current[current['Group_Name']!= "_Individual/Not belonging to any Group"]

#Temp_Pan
temp_pan = current.groupby(['Borrower_Customer_PAN','CIN'])['Total_Exposure'].sum()
temp_pan= temp_pan.to_frame()
temp_pan.reset_index()
temp_pan.columns=['te_sum']
# temp_pan.columns= ['borrower_customer_pan','cin','te_sum']

# Temp_pan_lei
temp_pan_lei=current.groupby(['Borrower_Customer_PAN','LEI'])['Total_Exposure'].sum()
temp_pan_lei= temp_pan_lei.to_frame()
temp_pan_lei.columns=['te_sum']
temp_pan_lei

#Temp_Cin
temp_pan= temp_pan.reset_index()
temp_cin = temp_pan[temp_pan.duplicated(['Borrower_Customer_PAN'], keep=False)]
temp_cin.columns= ['borrower_customer_pan','cin','te_sum']
temp_cin

# Temp_Cin1
temp_cin1 = temp_cin.groupby(['borrower_customer_pan'], sort=False)['te_sum'].max()
temp_cin1= temp_cin1.to_frame()
temp_cin1 = temp_cin1.reset_index()
temp_cin1.columns= ['borrower_customer_pan','MIN_TE']
temp_cin1

# Temp_pan_final
temp_cin['borrower_customer_pan'].isin(temp_cin1['borrower_customer_pan']).value_counts()
final_data =temp_cin.merge(temp_cin1,on =['borrower_customer_pan'])
temp_pan_final = final_data.loc[final_data["te_sum"] != final_data["MIN_TE"]]
temp_pan_final= temp_pan_final.drop('MIN_TE',axis=1)
temp_pan_final

#Temp_lei
temp_pan_lei= temp_pan_lei.reset_index()
temp_lei = temp_pan_lei[temp_pan_lei.duplicated(['Borrower_Customer_PAN'], keep=False)]
temp_lei.columns= ['borrower_customer_pan','lei','te_sum']
temp_lei

#Temp_Lei1
temp_lei1 = temp_lei.groupby(['borrower_customer_pan'], sort=False)['te_sum'].max()
temp_lei1= temp_lei1.to_frame()
temp_lei1.columns=['MIN_TE']
temp_lei1.reset_index()

# Temp_Lei_Final
temp_lei1= temp_lei1.reset_index()
temp_lei1.columns=['borrower_customer_pan', 'min_te']
temp_lei['borrower_customer_pan'].isin(temp_lei1['borrower_customer_pan']).value_counts()
final_data =temp_lei.merge(temp_lei1,on =['borrower_customer_pan'])
temp_lei_final = final_data.loc[final_data["te_sum"] != final_data["min_te"]]
temp_lei_final= temp_lei_final.drop('min_te',axis=1)
len(temp_lei_final)

#Temp_Pan_Same_Te
c1 = temp_cin
c2 = temp_cin
p1 = temp_pan_final
merge= c1.merge(p1,on =['borrower_customer_pan'])
Final_Data = c1.merge(merge.drop_duplicates(), on=['borrower_customer_pan'],how='left', indicator=True)
Output_Data =Final_Data[Final_Data['_merge']=='left_only']
Output_Data = Output_Data[['borrower_customer_pan','cin','te_sum']]
Output_Data = Output_Data[Output_Data.duplicated(['borrower_customer_pan'], keep=False)]
temp_pan_same = Output_Data
temp_pan_same

# Temp_Lei_Same_Te
c1 = temp_lei
c2 = temp_lei
p1 = temp_lei_final
merge= c1.merge(p1,on =['borrower_customer_pan'])
Final_Data = c1.merge(merge.drop_duplicates(), on=['borrower_customer_pan'],how='left', indicator=True)
Output_Data =Final_Data[Final_Data['_merge']=='left_only']
Output_Data = Output_Data[['borrower_customer_pan','lei','te_sum']]
Output_Data = Output_Data[Output_Data.duplicated(['borrower_customer_pan'], keep=False)]
temp_lei_same = Output_Data
temp_lei_same

# Temp_Default

temp_indefault= indefault
temp_outdefault= outdefault

temp_indefault= temp_indefault[['Bank Name', 'Date', 'Borrower Name', 'Borrower PAN', 'Date of Default','Funded Amount Outstanding as on Reporting Date', 'Total Amount Outstanding (Funded + Non-funded) as on Reporting Date']]
temp_outdefault= temp_outdefault[['Bank Name', 'Date', 'Borrower PAN', 'Date of Date of moving out of default']]

temp_indefault.columns=['Bank_Name', 'Date', 'Borrower_Name', 'Borrower_PAN', 'Date_of_Default','FAO', 'Total_Amount_Outstanding']
temp_outdefault.columns= ['Bank_Name', 'Date', 'Borrower_PAN', 'Date_of_moving_out_of_default']

left_join = temp_indefault.merge(temp_outdefault, on=['Borrower_PAN', 'Bank_Name'],how='inner')
Greater = left_join[left_join['Date_of_Default'] < left_join['Date_of_moving_out_of_default']]


Greater = Greater[['Borrower_PAN','Bank_Name','Date_x', 'Date_y']]
Greater.columns= ['Borrower_PAN', 'Bank_Name', 'date_of_default_in','Date_of_moving_out_of_default']
temp_default = Greater
temp_default.shape

(16666, 4)

In [40]:
#Temp_Borrower
Temp_Borrower = current[current['Group_Name']!= "_Individual/Not belonging to any Group"]

#Temp_Pan
temp_pan = current.groupby(['Borrower_Customer_PAN','CIN'])['Total_Exposure'].sum()
temp_pan= temp_pan.to_frame()
temp_pan.reset_index()
temp_pan.columns=['te_sum']
# temp_pan.columns= ['borrower_customer_pan','cin','te_sum']

# Temp_pan_lei
temp_pan_lei=current.groupby(['Borrower_Customer_PAN','LEI'])['Total_Exposure'].sum()
temp_pan_lei= temp_pan_lei.to_frame()
temp_pan_lei.columns=['te_sum']
temp_pan_lei

#Temp_Cin
temp_pan= temp_pan.reset_index()
temp_cin = temp_pan[temp_pan.duplicated(['Borrower_Customer_PAN'], keep=False)]
temp_cin.columns= ['borrower_customer_pan','cin','te_sum']
temp_cin

# Temp_Cin1
temp_cin1 = temp_cin.groupby(['borrower_customer_pan'], sort=False)['te_sum'].max()
temp_cin1= temp_cin1.to_frame()
temp_cin1 = temp_cin1.reset_index()
temp_cin1.columns= ['borrower_customer_pan','MIN_TE']
temp_cin1

# Temp_pan_final
temp_cin['borrower_customer_pan'].isin(temp_cin1['borrower_customer_pan']).value_counts()
final_data =temp_cin.merge(temp_cin1,on =['borrower_customer_pan'])
temp_pan_final = final_data.loc[final_data["te_sum"] != final_data["MIN_TE"]]
temp_pan_final= temp_pan_final.drop('MIN_TE',axis=1)
temp_pan_final

#Temp_lei
temp_pan_lei= temp_pan_lei.reset_index()
temp_lei = temp_pan_lei[temp_pan_lei.duplicated(['Borrower_Customer_PAN'], keep=False)]
temp_lei.columns= ['borrower_customer_pan','lei','te_sum']
temp_lei

#Temp_Lei1
temp_lei1 = temp_lei.groupby(['borrower_customer_pan'], sort=False)['te_sum'].max()
temp_lei1= temp_lei1.to_frame()
temp_lei1.columns=['MIN_TE']
temp_lei1.reset_index()

# Temp_Lei_Final
temp_lei1= temp_lei1.reset_index()
temp_lei1.columns=['borrower_customer_pan', 'min_te']
temp_lei['borrower_customer_pan'].isin(temp_lei1['borrower_customer_pan']).value_counts()
final_data =temp_lei.merge(temp_lei1,on =['borrower_customer_pan'])
temp_lei_final = final_data.loc[final_data["te_sum"] != final_data["min_te"]]
temp_lei_final= temp_lei_final.drop('min_te',axis=1)
len(temp_lei_final)

#Temp_Pan_Same_Te
c1 = temp_cin
c2 = temp_cin
p1 = temp_pan_final
merge= c1.merge(p1,on =['borrower_customer_pan'])
Final_Data = c1.merge(merge.drop_duplicates(), on=['borrower_customer_pan'],how='left', indicator=True)
Output_Data =Final_Data[Final_Data['_merge']=='left_only']
Output_Data = Output_Data[['borrower_customer_pan','cin','te_sum']]
Output_Data = Output_Data[Output_Data.duplicated(['borrower_customer_pan'], keep=False)]
temp_pan_same = Output_Data
temp_pan_same

# Temp_Lei_Same_Te
c1 = temp_lei
c2 = temp_lei
p1 = temp_lei_final
merge= c1.merge(p1,on =['borrower_customer_pan'])
Final_Data = c1.merge(merge.drop_duplicates(), on=['borrower_customer_pan'],how='left', indicator=True)
Output_Data =Final_Data[Final_Data['_merge']=='left_only']
Output_Data = Output_Data[['borrower_customer_pan','lei','te_sum']]
Output_Data = Output_Data[Output_Data.duplicated(['borrower_customer_pan'], keep=False)]
temp_lei_same = Output_Data
temp_lei_same

,borrower_customer_pan,lei,te_sum


In [41]:
#Temp_Non_Funded_Current
Asset_Data = ['Standard','Non-CDR Standard Restructured','Standard Restructured under CDR']
data = current.loc[current['Assets_Classification'].isin(Asset_Data)]
data = data[data['Template_Sheet_Name'] == 'Sec -1 -Exp-Largeborrow']
data1 = data[data['Special_Mention_Accounts'].isna()]
data10 = current.merge(data1, how='outer', indicator=True).query('_merge == "left_only"').drop('_merge', 1)
total_non_funded_amount = data10['Non_Funded_Amount_Outstanding_Total'].sum()
non_amount= data10.groupby(['Bank_Name','Special_Mention_Accounts'],dropna=False)['Non_Funded_Amount_Outstanding_Total'].sum().to_frame()
non_amount.reset_index(inplace=True)
non_amount.columns = ['Bank_Name', 'Special_Mention_Accounts','non_funded_amount']
non_amount["non_funded_amount_perc"] = np.round((non_amount['non_funded_amount']/total_non_funded_amount)*100,2)
non_amount= non_amount[['Bank_Name', 'Special_Mention_Accounts','non_funded_amount_perc']]
temp_non_funded_current= non_amount
temp_non_funded_current

#Temp_Non_Funded_Previous
Asset_Data = ['Standard','Non-CDR Standard Restructured','Standard Restructured under CDR']
data = previous.loc[previous['Assets_Classification'].isin(Asset_Data)]
data = data[data['Template_Sheet_Name'] == 'Sec -1 -Exp-Largeborrow']
data1 = data[data['Special_Mention_Accounts'].isna()]
data10 = previous.merge(data1, how='outer', indicator=True).query('_merge == "left_only"').drop('_merge', 1)
total_non_funded_amount = data10['Non_Funded_Amount_Outstanding_Total'].sum()
non_amount= data10.groupby(['Bank_Name','Special_Mention_Accounts'],dropna=False)['Non_Funded_Amount_Outstanding_Total'].sum().to_frame()
non_amount.reset_index(inplace=True)
non_amount.columns = ['Bank_Name', 'Special_Mention_Accounts','non_funded_amount']
non_amount["non_funded_amount_perc"] = np.round((non_amount['non_funded_amount']/total_non_funded_amount)*100,2)
non_amount= non_amount[['Bank_Name', 'Special_Mention_Accounts','non_funded_amount_perc']]
temp_non_funded_previous= non_amount
temp_non_funded_previous

C:\Users\admin\AppData\Local\Temp\ipykernel_11848\3377207404.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data10 = current.merge(data1, how='outer', indicator=True).query('_merge == "left_only"').drop('_merge', 1)
C:\Users\admin\AppData\Local\Temp\ipykernel_11848\3377207404.py:21: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data10 = previous.merge(data1, how='outer', indicator=True).query('_merge == "left_only"').drop('_merge', 1)


,Bank_Name,Special_Mention_Accounts,non_funded_amount_perc
0,AMERICAN EXPRESS BANKING CORP.,SMA-0,0.00
1,AMERICAN EXPRESS BANKING CORP.,SMA-1,0.00
2,AMERICAN EXPRESS BANKING CORP.,SMA-2,0.00
3,AMERICAN EXPRESS BANKING CORP.,NaN,0.00
4,AU SMALL FINANCE BANK LIMITED,SMA-0,0.00
...,...,...,...
198,WOORI BANK,NaN,0.00
199,YES BANK LTD.,SMA-0,1.27
200,YES BANK LTD.,SMA-1,0.34
201,YES BANK LTD.,SMA-2,0.22


In [42]:
# with pd.ExcelWriter("Temp_Output.xlsx") as writer:   
#     # use to_excel function and specify the sheet_name and index
#     # to store the dataframe in specified sheet
    
#     Temp_Borrower.to_excel(writer, sheet_name="Temp_Borrower", index=False) 
#     temp_pan.to_excel(writer, sheet_name="temp_pan", index=False) 
    
#     temp_pan_lei.to_excel(writer, sheet_name="temp_pan_lei", index=False) 
#     temp_cin.to_excel(writer, sheet_name="temp_cin", index=False) 
    
#     temp_cin1.to_excel(writer, sheet_name="temp_cin1", index=False) 
#     temp_pan_final.to_excel(writer, sheet_name="temp_pan_final", index=False) 
    
#     temp_lei.to_excel(writer, sheet_name="temp_lei", index=False) 
#     temp_lei1.to_excel(writer, sheet_name="temp_lei1", index=False) 
    
#     temp_lei_final.to_excel(writer, sheet_name="temp_lei_final", index=False) 
#     temp_pan_same.to_excel(writer, sheet_name="temp_pan_same", index=False) 
    
#     temp_lei_same.to_excel(writer, sheet_name="temp_lei_same", index=False) 
#     temp_default.to_excel(writer, sheet_name="temp_default", index=False) 
    
#     temp_lei_final.to_excel(writer, sheet_name="temp_lei_final", index=False) 
#     temp_pan_same_te.to_excel(writer, sheet_name="temp_pan_same_te", index=False) 
    
#     temp_non_funded_current.to_excel(writer, sheet_name="temp_non_funded_current", index=False) 
#     temp_non_funded_previous.to_excel(writer, sheet_name="temp_non_funded_previous", index=False) 

#     writer.save()

In [43]:
CIN = current[(current['Borrower_Customer_PAN'].str[3:4] == 'C')]
CIN = CIN[(CIN['Borrower_Customer_PAN'].str[0:3] != 'RBI')]
CIN =  CIN[(CIN['CIN'].isna())]
CIN = CIN[CIN['Template_Sheet_Name'] == 'Sec -1 -Exp-Largeborrow']
CIN = CIN[["Date","Template_Sheet_Name","Borrower_Customer_PAN","CIN","Borrower_Customer_Name","Assets_Classification","Date_of_NPA_Classification","Bank_Name","Total_Exposure"]]
CIN.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
CIN

# CIN_INDIVIDUAL
Cin_Individual= current[~(current['CIN'].isna())]
Cin_Individual = Cin_Individual[(Cin_Individual['Borrower_Customer_PAN'].str[3:4] == 'P')]
Cin_Individual = Cin_Individual[["Date","Template_Sheet_Name","Borrower_Customer_PAN","CIN","LEI","Borrower_Customer_Name","Assets_Classification","Bank_Name","Total_Exposure"]]
Cin_Individual.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
Cin_Individual

#CIN_RBI
Cin_Rbi = current[(current['Borrower_Customer_PAN'].str[0:3] == 'RBI')]
Cin_Rbi= Cin_Rbi[~(Cin_Rbi['CIN'].isna())]
Cin_Rbi = Cin_Rbi[["Date","Template_Sheet_Name","Borrower_Customer_PAN","CIN","LEI","Borrower_Customer_Name","Assets_Classification","Bank_Name","Total_Exposure"]]
Cin_Rbi.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
Cin_Rbi

#CIN_PAN_COMBO
temp_pan_final.columns= ['Borrower_Customer_PAN', 'CIN', 'te_sum']
temp =current.merge(temp_pan_final,on =['Borrower_Customer_PAN','CIN'])
temp_pan_same.columns=['Borrower_Customer_PAN', 'CIN', 'te_sum']
temp1 =current.merge(temp_pan_same,on =['Borrower_Customer_PAN','CIN'])
CIN_PAN_Combo =pd.merge(temp,temp1, how='outer')
CIN_PAN_Combo = CIN_PAN_Combo[["Date","Template_Sheet_Name","Borrower_Customer_PAN","CIN","LEI","Borrower_Customer_Name","Assets_Classification","Bank_Name","Total_Exposure"]]
CIN_PAN_Combo =CIN_PAN_Combo.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
len(CIN_PAN_Combo)

#LEI_TE
curr_month_mod = current[(current['Borrower_Customer_PAN'].str[3] != 'P')]
curr_month_mod = curr_month_mod[(curr_month_mod['Borrower_Customer_PAN'].str[0:3] != 'RBI')]
pan = curr_month_mod.groupby('Borrower_Customer_PAN', as_index=False)["Total_Exposure"].sum()
df_test= pan[pan['Total_Exposure']>=50]
Final_output = curr_month_mod.loc[curr_month_mod["Borrower_Customer_PAN"].isin(df_test["Borrower_Customer_PAN"].unique().tolist())]
final_df = Final_output[Final_output['LEI'].isnull()]
Output = final_df[final_df['Template_Sheet_Name'] == 'Sec -1 -Exp-Largeborrow']
Lei_Te = Output
Lei_Te = Lei_Te[["Date","Template_Sheet_Name","Borrower_Customer_PAN","LEI","Borrower_Customer_Name","Assets_Classification","Bank_Name","Total_Exposure"]]
Lei_Te =Lei_Te.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
len(Lei_Te)

# LEI_INDIVIDUAL
LEI_Individual = current[(current['Borrower_Customer_PAN'].str[3:4] == 'P')]
LEI_Individual= LEI_Individual[~(LEI_Individual['LEI'].isna())]
LEI_Individual = LEI_Individual[(LEI_Individual['Borrower_Customer_PAN'].str[3:4] == 'P')]
LEI_Individual = LEI_Individual[~(LEI_Individual['Borrower_Customer_PAN'].str.contains("RBI"))]
LEI_Individual.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
LEI_Individual = LEI_Individual[["Date","Template_Sheet_Name","Borrower_Customer_PAN","CIN","LEI","Borrower_Customer_Name","Assets_Classification","Bank_Name","Total_Exposure"]]
LEI_Individual =  LEI_Individual.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
len(LEI_Individual)

# LEI_PAN_Combo
temp_lei_final.columns=['Borrower_Customer_PAN', 'LEI', 'te_sum']
temp_lei_same.columns=['Borrower_Customer_PAN', 'LEI', 'te_sum']
merge1 = pd.merge(current,temp_lei_final, how= 'inner', on =['Borrower_Customer_PAN','LEI'])
merge1 = merge1[["Date", "Template_Sheet_Name", "Borrower_Customer_PAN","CIN", "LEI", "Borrower_Customer_Name", "Assets_Classification", "Bank_Name", "Total_Exposure"]]
merge2 = pd.merge(current,temp_lei_same, how= 'inner', on =['Borrower_Customer_PAN','LEI'])
merge2 = merge2[["Date", "Template_Sheet_Name", "Borrower_Customer_PAN","CIN", "LEI", "Borrower_Customer_Name", "Assets_Classification", "Bank_Name", "Total_Exposure"]]
LEI_PAN_Combo = pd.concat([merge1, merge2], join='inner', ignore_index=True)
LEI_PAN_Combo.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
len(LEI_PAN_Combo)

236

In [44]:
# External_Rating_Changes

t1= two_month
c1= current
p1= previous

merged = c1.merge(p1,on=['Borrower_Customer_PAN','Bank_Name']).merge(t1,on=['Borrower_Customer_PAN','Bank_Name'])
merged = merged[merged["External_Rating_x"] != merged["External_Rating_y"]]
merged.dropna(subset= ["External_Rating_x"],inplace = True)
merged = merged[merged["External_Rating_y"] != merged["External_Rating"]]
merged.dropna(subset= ["External_Rating_y"],inplace = True)
merged.dropna(subset= ["External_Rating"],inplace = True)
merged = merged.drop_duplicates()
merged2 = c1.merge(p1,on=['Borrower_Customer_PAN','Bank_Name']).merge(t1,on=['Borrower_Customer_PAN','Bank_Name'])
merged2 = c1.merge(p1,on=['Borrower_Customer_PAN','Bank_Name']).merge(t1,on=['Borrower_Customer_PAN','Bank_Name'])
merged2 = merged2[(merged2['External_Rating_y'].isna()) & (~merged2['External_Rating_x'].isna()) & (~merged2['External_Rating'].isna())]
External_rating_changes = pd.concat([merged, merged2])
External_rating_changes.reset_index(drop=True)
External_rating_changes = External_rating_changes[["Date_x", "Template_Sheet_Name_x", "Borrower_Customer_PAN","Borrower_Customer_Name_x","Assets_Classification_x", "External_Rating_x", "Bank_Name","Total_Exposure_x","External_Rating_y", "External_Rating"]]
External_rating_changes.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
External_rating_changes.drop_duplicates()
External_rating_changes = External_rating_changes[['Date_x', 'Template_Sheet_Name_x', 'Borrower_Customer_PAN','Borrower_Customer_Name_x', 'Assets_Classification_x',
       'External_Rating_x', 'Bank_Name', 'Total_Exposure_x','External_Rating_y', 'External_Rating']]
External_rating_changes.columns= ['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN','Borrower_Customer_Name', 'Assets_Classification',
       'External_Rating', 'Bank_Name', 'Total_Exposure','Previous_Month_External_Rating', 'TwoMonth_back_External_Rating']
External_Rating_Changes = External_rating_changes

In [45]:
# Lead_Bank_Reporting
current_new = current.dropna(subset=['Lead_Bank_Name'])
previous_new = previous.dropna(subset=['Lead_Bank_Name'])
Two_Months_Back_new = two_month.dropna(subset=['Lead_Bank_Name'])
merge1 = pd.merge(current_new,previous_new, how= 'inner', on =['Borrower_Customer_PAN','Bank_Name'])
merge2 = pd.merge(merge1,Two_Months_Back_new, how= 'inner', on =['Borrower_Customer_PAN','Bank_Name'])
Lead_Bank = merge2[(merge2['Lead_Bank_Name_x'] != merge2['Lead_Bank_Name_y'])]
Lead_Bank_Reporting = Lead_Bank[(Lead_Bank['Lead_Bank_Name_y'] != Lead_Bank['Lead_Bank_Name'])]
Lead_Bank_Reporting = merge2[(merge2['Lead_Bank_Name_x']!= merge2['Lead_Bank_Name_y']) & (merge2['Lead_Bank_Name_y']!= merge2['Lead_Bank_Name'])]
Lead_Bank_Reporting = Lead_Bank_Reporting[["Date_x", "Template_Sheet_Name_x", "Borrower_Customer_PAN","Borrower_Customer_Name_x", "Assets_Classification_x", "Bank_Name", "Lead_Bank_Name_x","Total_Investment_Exposure","Total_Exposure_x",'Lead_Bank_Name_y','Lead_Bank_Name']]
Lead_Bank_Reporting.columns= ['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN','Borrower_Customer_Name', 'Assets_Classification', 'Bank_Name','Lead_Bank_Name','Total_Investment_Exposure', 'Total_Exposure','Previous_Month_Lead_Bank_Name','TwoMonth_back_Lead_Bank_Name']
Lead_Bank_Reporting.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
len(Lead_Bank_Reporting)

13

In [46]:
# Lead_Bank_Exclude
current_new = current
Lead_Bank_Exclude = current_new[current_new['Lead_Bank_Name'].isin(["Allahabad Bank", "Andhra Bank", "Corporation Bank", "Dena Bank", 
 "Oriental Bank Of Commerce" ,"Syndicate Bank", "United Bank Of India","Vijaya Bank","LAKSHMI VILAS BANK LTD"])]
Lead_Bank_Exclude = Lead_Bank_Exclude[["Date", "Template_Sheet_Name", "Borrower_Customer_PAN","Borrower_Customer_Name", "Assets_Classification", "Bank_Name", "Lead_Bank_Name", "Total_Exposure"]]
Lead_Bank_Exclude.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
Lead_Bank_Exclude= Lead_Bank_Exclude[['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN','Borrower_Customer_Name', 'Assets_Classification', 'Bank_Name','Lead_Bank_Name', 'Total_Exposure']]
Lead_Bank_exclude = Lead_Bank_Exclude

In [47]:
# Industry_Classification-  Data doesn't match as it doesn't matches exact name,  ex: CAPITAL in CAPITALS,....
current_new = current
current_1= current_new[current_new['Industry_Code']== 99999] 
Industry_Classification = current_1[(current_1['Borrower_Customer_Name'].str.contains('CAPITAL')) | (current_1['Borrower_Customer_Name'].str.contains('FINANCE')) | (current_1['Borrower_Customer_Name'].str.contains('FUND')) | (current_1['Borrower_Customer_Name'].str.contains('JEWELLERY')) | (current_1['Borrower_Customer_Name'].str.contains('HOSPITAL')) | (current_1['Borrower_Customer_Name'].str.contains('HEALTH')) | (current_1['Borrower_Customer_Name'].str.contains('STEEL')) | (current_1['Borrower_Customer_Name'].str.contains('TELECOM')) | (current_1['Borrower_Customer_Name'].str.contains('TOURISM')) | (current_1['Borrower_Customer_Name'].str.contains('AIRLINE')) | (current_1['Borrower_Customer_Name'].str.contains('CONSTRUCTION')) | (current_1['Borrower_Customer_Name'].str.contains('ELECTRICITY')) | (current_1['Borrower_Customer_Name'].str.contains('TRANSPORT')) | (current_1['Borrower_Customer_Name'].str.contains('HIGHWAYS')) | (current_1['Borrower_Customer_Name'].str.contains('EDUCATION'))] 
Industry_Classification.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
Industry_Classification = Industry_Classification[["Date", "Template_Sheet_Name", "Borrower_Customer_PAN","Borrower_Customer_Name", "Assets_Classification", "Bank_Name","Industry_Name", "Industry_Code", "Total_Exposure"]]
len(Industry_Classification)

# Industry_code_change
current_new = current
previous_new = previous
Two_Months_Back_new = two_month
merge1 = pd.merge(current_new,previous_new, how= 'inner', on =['Borrower_Customer_PAN','Bank_Name'])
merge2 = pd.merge(merge1,Two_Months_Back_new, how= 'inner', on =['Borrower_Customer_PAN','Bank_Name'])
Industry_code_change = merge2[(merge2['Industry_Code_x']!= merge2['Industry_Code_y']) & (merge2['Industry_Code_y']!= merge2['Industry_Code'])]
Industry_code_change = Industry_code_change[["Date_x", "Template_Sheet_Name_x", "Borrower_Customer_PAN","Borrower_Customer_Name_x","Industry_Code_x", "Bank_Name", "Total_Exposure_x","Industry_Code_y", "Industry_Code", "Template_Sheet_Name_y", "Template_Sheet_Name"]]
Industry_code_change.columns = ["Date", "Template_Sheet_Name", "Borrower_Customer_PAN","Borrower_Customer_Name","Industry_Code", "Bank_Name", "Total_Exposure","Industry_Code_Previous_Month", "Industry_Code_Two_Months_Back", "Template_Sheet_Name_Previous_Month", "Template_Sheet_Name_Two_Months_Back"]

Industry_code_change.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
len(Industry_code_change)

# Borrower_Group
c1= current
t1= Temp_Borrower
temp = pd.concat([c1,t1],axis = 1,keys= ["c1","t1"])
temp = temp[temp['c1']['Borrower_Customer_PAN'].isin(temp['t1']['Borrower_Customer_PAN'])]
temp = temp[temp["c1"]["Group_Name"] == "_Individual/Not belonging to any Group"]
Borrower_Group = temp["c1"]
Borrower_Group= Borrower_Group[['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN', 'Borrower_Customer_Name', 'Group_Name', 'Bank_Name','Total_Exposure']]
Borrower_Group.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
len(Borrower_Group)

10811

In [48]:
# Under_reporting
c1= current
p1= previous
t1= two_month
merge = pd.merge(c1,t1,how = "inner", on=['Borrower_Customer_PAN','Bank_Name'])
c_total_Ex = merge[merge['Total_Exposure_x']>20]
Exist= pd.merge(c1,t1, on=['Borrower_Customer_PAN','Bank_Name'])
Not_Exist= pd.merge(c1,p1, on=['Borrower_Customer_PAN','Bank_Name'])
Final_Data = c_total_Ex.merge(Not_Exist.drop_duplicates(), on=['Borrower_Customer_PAN','Bank_Name'],how='left', indicator=True)
Under_reporting = Final_Data[Final_Data['_merge']=='left_only']
Under_reporting=Under_reporting[['Date_x_x', 'Template_Sheet_Name_x_x', 'Borrower_Customer_PAN', 'Borrower_Customer_Name_x_x','Assets_Classification_x_x','Bank_Name','Total_Exposure_x_x','Total_Exposure_y_x']]
Under_reporting.columns=['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN', 'Borrower_Customer_Name','Assets_Classification','Bank_Name','current_month_total_exposure','2months_back_total_exposure']
Under_reporting.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
Under_reporting

,Date,Template_Sheet_Name,Borrower_Customer_PAN,Borrower_Customer_Name,Assets_Classification,Bank_Name,current_month_total_exposure,2months_back_total_exposure
99,2023/01/31 00:00:00,Sec -1 -Exp-Largeborrow,AAAAI0038F,ICICI PRUDENTIAL MUTUAL FUND,Standard,UNION BANK OF INDIA,349.982500,249.987500
102,2023/01/31 00:00:00,Sec -1 -Exp-Largeborrow,AAAAI0038F,ICICI PRUDENTIAL MUTUAL FUND,Standard,IDBI BANK LIMITED,139.993000,149.992500
286,2023/01/31 00:00:00,Sec -1 -Exp-Largeborrow,AAAAT3400C,THE KRISHNA SAHAKARI SAKKARE KARKHANE NIYAMIT,Standard,STATE BANK OF INDIA,37.377600,17.621300
652,2023/01/31 00:00:00,Sec -1 -Exp-Largeborrow,AAACA4710A,AGARWAL FASTNERS PVT LTD,Standard,KOTAK MAHINDRA BANK LTD.,20.999428,20.804164
715,2023/01/31 00:00:00,Sec -1 -Exp-Largeborrow,AAACA5424H,AMAZER INVESTMENT AND FINANCE LIMITED,Standard,UNITY SMALL FINANCE BANK LIMITED,29.000000,29.000000
...,...,...,...,...,...,...,...,...
57500,2023/01/31 00:00:00,Sec -1 -Exp-Largeborrow,RBICS0124F,SVIZERA HOLDINGS B.V.,Loss,BANK OF INDIA,66.432500,66.029100
57563,2023/01/31 00:00:00,Sec -1 -Exp-Largeborrow,RBICS1134F,SOLAR OVERSEAS MAURITIUS LTD,Standard,INDUSIND BANK LTD,43.010600,42.820000
57770,2023/01/31 00:00:00,Sec -1 -Exp-Largeborrow,RBICT1084F,THIESS GROUP FINANCE PTY LTD,Standard,BANK OF BARODA,98.011300,119.798400
57771,2023/01/31 00:00:00,Sec -1 -Exp-Largeborrow,RBICT1084F,THIESS GROUP FINANCE PTY LTD,Standard,STATE BANK OF INDIA,161.018500,196.775800


In [49]:
# Miscellaneous
current_new = current[current['Template_Sheet_Name']== "Sec -1 -Exp-Largeborrow"]
previous_new= previous[previous['Template_Sheet_Name']=="Sec -2 WriteOff"]
miscellaneous =current_new.merge(previous_new,on =['Borrower_Customer_PAN','Bank_Name'])
miscellaneous.drop([ 'CIN_x','LEI_x','Group_Name_x', 'Industry_Code_x', 'External_Rating_x','Borrower_Classified_as_Fraud_x', 'Special_Mention_Accounts_x','Date_of_SMA_Classification_x','Bank_Group_Level3_x','Date_y', 'CIN_y', 'LEI_y', 'Borrower_Customer_Name_y','Assets_Classification_y', 'Date_of_NPA_Classification_y','Group_Name_y', 'Industry_Code_y', 'Amount_technically_prudentially_y','External_Rating_y', 'Borrower_Classified_as_Fraud_y','Special_Mention_Accounts_y', 'Date_of_SMA_Classification_y','Funded_Amount_Outstanding_Total_y','Non_Funded_Amount_Outstanding_Total_y', 'Total_Credit_Exposure_y','Bank_Group_Level3_y', 'Lead_Bank_Name_y','Total_Investment_Exposure_y','Total_Exposure_y'],inplace=True,axis=1)
miscellaneous.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
miscellaneous = miscellaneous[['Date_x', 'Template_Sheet_Name_x', 'Borrower_Customer_PAN','Borrower_Customer_Name_x', 'Assets_Classification_x','Date_of_NPA_Classification_x',
       'Amount_technically_prudentially_x','Funded_Amount_Outstanding_Total_x','Non_Funded_Amount_Outstanding_Total_x', 'Total_Credit_Exposure_x','Bank_Name', 'Lead_Bank_Name_x', 'Total_Investment_Exposure_x',
       'Total_Exposure_x', 'Template_Sheet_Name_y']]
miscellaneous.columns = ['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN','Borrower_Customer_Name', 'Assets_Classification','Date_of_NPA_Classification',
       'Amount_technically_prudentially','Funded_Amount_Outstanding_Total','Non_Funded_Amount_Outstanding_Total', 'Total_Credit_Exposure','Bank_Name', 'Lead_Bank_Name', 'Total_Investment_Exposure',
       'Total_Exposure', 'Previous_Month_Template_sheet']
len(miscellaneous)

# Bank_Level_SMA
# merge_new= temp_non_funded_current.merge(temp_non_funded_previous, how = "inner",on=['Special_Mention_Accounts','Bank_Name'])
# merge_all = merge_new[(merge_new['non_funded_amount_perc_y']-merge_new['non_funded_amount_perc_x']>4) | (merge_new['non_funded_amount_perc_y']-merge_new['non_funded_amount_perc_x']<-4)]
# Bank_level_SMA = merge_all
# Bank_level_SMA.columns= ['Bank_Name','Special_Mention_Accounts','non_funded_amount_perc_current','non_funded_amount_perc_previous']

# SMA_Reporting
current_new= current
previous_new = previous
merge=current_new.merge(previous_new,on =['Borrower_Customer_PAN','Bank_Name'])
merge= merge[(merge['Assets_Classification_y']== 'Sub-standard') | (merge['Assets_Classification_y']== 'Substandard Restructured') | (merge['Assets_Classification_y']== 'Doubtful 1') | (merge['Assets_Classification_y']== 'Doubtful 2') | (merge['Assets_Classification_y']== 'Doubtful 3') | (merge['Assets_Classification_y']== 'Doubtful Restructured') | (merge['Assets_Classification_y']== 'Loss')]
SMA_Reporting =  merge[(merge['Special_Mention_Accounts_x']== 'SMA-1') | (merge['Special_Mention_Accounts_x']== 'SMA-2')]
SMA_Reporting = SMA_Reporting[['Date_x', 'Template_Sheet_Name_x', 'Borrower_Customer_PAN','Borrower_Customer_Name_x', 'Assets_Classification_x','Special_Mention_Accounts_x', 'Date_of_SMA_Classification_x','Bank_Name', 'Total_Exposure_x', 'Assets_Classification_y']]
SMA_Reporting.columns= ['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN','Borrower_Customer_Name', 'Assets_Classification','Special_Mention_Accounts', 'Date_of_SMA_Classification','Bank_Name', 'Total_Exposure', 'Previous_Month_Assets_Classification']
SMA_Reporting.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
len(SMA_Reporting)

# NPA_Classification_i
current1= current
previous1 = previous
c1= current1[(current1['Assets_Classification']== 'Sub-standard') | (current1['Assets_Classification']== 'Substandard Restructured')]
p1=  previous1[(previous1['Assets_Classification']== 'Doubtful 1') | (previous1['Assets_Classification']== 'Doubtful Restructured') | (previous1['Assets_Classification']== 'Doubtful 2') | (previous1['Assets_Classification']== 'Doubtful 3') | (previous1['Assets_Classification']== 'Loss')]
merge1 = c1.merge(p1,on =['Borrower_Customer_PAN','Bank_Name'])
merge1= merge1[['Date_x', 'Template_Sheet_Name_x', 'Borrower_Customer_PAN','Borrower_Customer_Name_x', 'Assets_Classification_x','Date_of_NPA_Classification_x', 'Amount_technically_prudentially_x','Bank_Name', 'Bank_Group_Level3_x', 'Total_Exposure_x','Assets_Classification_y']]


merge1.columns= ['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN','Borrower_Customer_Name', 'Assets_Classification','Date_of_NPA_Classification', 'Amount_technically_prudentially','Bank_Name', 'Bank_Group_Level3', 'Total_Exposure','Previous_Month_Assets_Classification']
current2= current
previous2= previous
merge2 = current2.merge(previous2,on =['Borrower_Customer_PAN','Bank_Name'])
merge2= merge2[(merge2['Assets_Classification_x']== 'Doubtful 1')]
merge2=  merge2[(merge2['Assets_Classification_y']== 'Doubtful 2') | (merge2['Assets_Classification_y']== 'Doubtful 3') | (merge2['Assets_Classification_y']== 'Loss')]
merge2= merge2[['Date_x', 'Template_Sheet_Name_x', 'Borrower_Customer_PAN','Borrower_Customer_Name_x', 'Assets_Classification_x','Date_of_NPA_Classification_x', 'Amount_technically_prudentially_x','Bank_Name', 'Bank_Group_Level3_x', 'Total_Exposure_x','Assets_Classification_y']]
merge2.columns= ['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN','Borrower_Customer_Name', 'Assets_Classification','Date_of_NPA_Classification', 'Amount_technically_prudentially','Bank_Name', 'Bank_Group_Level3', 'Total_Exposure','Previous_Month_Assets_Classification']

current3= current
previous3= previous
merge3 = current3.merge(previous3,on =['Borrower_Customer_PAN','Bank_Name'])
merge3= merge3[(merge3['Assets_Classification_x']== 'Doubtful 3') | (merge3['Assets_Classification_x']== 'Doubtful Restructured')]
merge3=  merge3[(merge3['Assets_Classification_y']== 'Loss')]
merge3= merge3[['Date_x', 'Template_Sheet_Name_x', 'Borrower_Customer_PAN','Borrower_Customer_Name_x', 'Assets_Classification_x','Date_of_NPA_Classification_x', 'Amount_technically_prudentially_x','Bank_Name', 'Bank_Group_Level3_x', 'Total_Exposure_x','Assets_Classification_y']]
merge3.columns= ['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN','Borrower_Customer_Name', 'Assets_Classification','Date_of_NPA_Classification', 'Amount_technically_prudentially','Bank_Name', 'Bank_Group_Level3', 'Total_Exposure','Previous_Month_Assets_Classification']

current1= current1[current1['Assets_Classification']== 'Doubtful 2']
previous1= previous1[(previous1['Assets_Classification'] == 'Doubtful 3')| (previous1['Assets_Classification'] == 'Loss')]
merge10 = current1.merge(previous1,on =['Borrower_Customer_PAN','Bank_Name'])
merge10.columns
merge10= merge10[['Date_x', 'Template_Sheet_Name_x', 'Borrower_Customer_PAN','Borrower_Customer_Name_x', 'Assets_Classification_x','Date_of_NPA_Classification_x', 'Amount_technically_prudentially_x','Bank_Name', 'Bank_Group_Level3_x', 'Total_Exposure_x','Assets_Classification_y']]
merge10.columns= ['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN','Borrower_Customer_Name', 'Assets_Classification','Date_of_NPA_Classification', 'Amount_technically_prudentially','Bank_Name', 'Bank_Group_Level3', 'Total_Exposure','Previous_Month_Assets_Classification']

NPA_Class_i= pd.concat([merge1, merge2, merge3,merge10], join='outer', ignore_index=True)
len(NPA_Class_i)

# NPA_Classification_ii 
current_new= current
previous_new= previous
mapping_table_new = mapping_table
current_new = current_new[(current_new['Assets_Classification']== 'Sub-standard') | (current_new['Assets_Classification']== 'Substandard Restructured') | (current_new['Assets_Classification']== 'Doubtful Restructured') | (current_new['Assets_Classification']== 'Doubtful 1')| (current_new['Assets_Classification']== 'Doubtful 2') | (current_new['Assets_Classification']== 'Doubtful 3') | (current_new['Assets_Classification']== 'Loss')]
current_new['Assets_Classification'].unique()
mapping_table_new = mapping_table_new[(mapping_table_new['Ext_Rating'].str.contains('BBB')) | (mapping_table_new['Ext_Rating'].str[0] == 'A')] 
mapping_table_new
NPA_Class_ii = pd.merge(current_new,mapping_table_new,on =['External_Rating'])
NPA_Class_ii.columns
NPA_Class_ii= NPA_Class_ii[['Date','Template_Sheet_Name','Borrower_Customer_PAN','Borrower_Customer_Name','Assets_Classification','Date_of_NPA_Classification','External_Rating','Bank_Name','Total_Exposure']]
NPA_Class_ii.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
len(NPA_Class_ii)

# NPA_Classification_iii

temp1 = current_new.merge(previous_new,on =['Borrower_Customer_PAN','Bank_Name'])
temp1.columns
temp2 = temp1.merge(Two_Months_Back_new,on =['Borrower_Customer_PAN','Bank_Name'])
temp2.columns
output= temp2[(temp2['Assets_Classification_x']== 'Sub-standard') | (temp2['Assets_Classification_x']== 'Substandard Restructured') | (temp2['Assets_Classification_x']== 'Doubtful Restructured') | (temp2['Assets_Classification_x']== 'Doubtful 1')| (temp2['Assets_Classification_x']== 'Doubtful 2') | (temp2['Assets_Classification_x']== 'Doubtful 3') | (temp2['Assets_Classification_x']== 'Loss')]
output= output[(output['Assets_Classification_y']== 'Standard') | (output['Assets_Classification_y']== 'Standard Restructured under CDR') | (output['Assets_Classification_y']== 'Non-CDR Standard Restructured')]
output= output[(output['Assets_Classification']== 'Sub-standard') | (output['Assets_Classification']== 'Substandard Restructured') | (output['Assets_Classification']== 'Doubtful Restructured') | (output['Assets_Classification']== 'Doubtful 1')| (output['Assets_Classification']== 'Doubtful 2') | (output['Assets_Classification']== 'Doubtful 3') | (output['Assets_Classification']== 'Loss')]
output= output[['Date_x','Template_Sheet_Name_x','Borrower_Customer_PAN','Borrower_Customer_Name_x','Assets_Classification_x','Date_of_NPA_Classification_x','Bank_Name','Total_Exposure_x','Assets_Classification_y','Assets_Classification']]
output.columns = ['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN','Borrower_Customer_Name', 'Assets_Classification','Date_of_NPA_Classification', 'Bank_Name', 'Total_Exposure','Previous_Month_Assets_Classification', 'TwoMonth_back_Assets_Classification']
NPA_Class_iii = output
NPA_Class_iii.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
len(NPA_Class_iii)

# NPA_Classification_iv

from datetime import datetime as dt
from datetime import timedelta
from dateutil.relativedelta import relativedelta  
current_new= current

current_new["Date"] = current_new["Date"].astype('datetime64[ns]')
current_new["dateadd"] = current_new["Date"] + timedelta(days=-364)
current_new["Date_of_NPA_Classification"] = pd.to_datetime(current_new["Date_of_NPA_Classification"]).dt.strftime('%Y-%m-%d')
current_new= current_new[current_new["Date_of_NPA_Classification"] < current_new["dateadd"]]
current_new= current_new[(current_new['Assets_Classification']== 'Sub-standard') | (current_new['Assets_Classification']== 'Substandard Restructured')]
current_new1= current
current_new1["dateadd"] = current_new1["Date"] + timedelta(days=-728)
current_new1["Date_of_NPA_Classification"] = pd.to_datetime(current_new1["Date_of_NPA_Classification"]).dt.strftime('%Y-%m-%d')
current_new1= current_new1[current_new1["Date_of_NPA_Classification"] < current_new1["dateadd"]]
current_new1= current_new1[(current_new1['Assets_Classification']== 'Doubtful 1')]

current_new2= current
current_new2["Date"] = current_new2["Date"].astype('datetime64[ns]')
current_new2["dateadd"] = current_new2["Date"] + timedelta(days=-1460)
current_new2["Date_of_NPA_Classification"] = pd.to_datetime(current_new2["Date_of_NPA_Classification"]).dt.strftime('%Y-%m-%d')
current_new2= current_new2[current_new2["Date_of_NPA_Classification"] < current_new2["dateadd"]]
current_new2= current_new2[(current_new2['Assets_Classification']== 'Doubtful 2')]
NPA_Class_iv= pd.concat([current_new, current_new1, current_new2], join='inner', ignore_index=True)
NPA_Class_iv= NPA_Class_iv[['Date','Template_Sheet_Name','Borrower_Customer_PAN','Borrower_Customer_Name','Assets_Classification','Date_of_NPA_Classification','Bank_Name','Bank_Group_Level3','Lead_Bank_Name','Total_Investment_Exposure','Total_Exposure']]
NPA_Class_iv.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
NPA_Class_iv

# NPA_Classification_v
current_new= current
previous_new= previous
Two_Months_Back_new = two_month

temp =current_new.merge(previous_new,on =['Borrower_Customer_PAN','Bank_Name'])
temp =temp.merge(Two_Months_Back_new,on =['Borrower_Customer_PAN','Bank_Name'])
len(temp)
temp
merge= temp[(temp['Assets_Classification_x']== 'Sub-standard') | (temp['Assets_Classification_x']== 'Substandard Restructured') | (temp['Assets_Classification_x']== 'Doubtful 1') | (temp['Assets_Classification_x']== 'Doubtful 2') | (temp['Assets_Classification_x']== 'Doubtful 3') | (temp['Assets_Classification_x']== 'Doubtful Restructured') | (temp['Assets_Classification_x']== 'Loss')]
merge= merge[(merge['Assets_Classification_y']== 'Sub-standard') | (merge['Assets_Classification_y']== 'Substandard Restructured') | (merge['Assets_Classification_y']== 'Doubtful 1') | (merge['Assets_Classification_y']== 'Doubtful 2') | (merge['Assets_Classification_y']== 'Doubtful 3') | (merge['Assets_Classification_y']== 'Doubtful Restructured') | (merge['Assets_Classification_y']== 'Loss')]
merge= merge[(merge['Assets_Classification']== 'Sub-standard') | (merge['Assets_Classification']== 'Substandard Restructured') | (merge['Assets_Classification']== 'Doubtful 1') | (merge['Assets_Classification']== 'Doubtful 2') | (merge['Assets_Classification']== 'Doubtful 3') | (merge['Assets_Classification']== 'Doubtful Restructured') | (merge['Assets_Classification']== 'Loss')]
merge
merge= merge[merge['Assets_Classification_x'] != merge['Assets_Classification_y']]
merge
merge= merge[merge['Assets_Classification_y'] != merge['Assets_Classification']]
merge.count()

NPA_Class_v= merge
NPA_Class_v= NPA_Class_v[['Date_x', 'Template_Sheet_Name_x', 'Borrower_Customer_PAN',
       'Borrower_Customer_Name_x', 'Assets_Classification_x','Date_of_NPA_Classification_x', 
        'Bank_Name', 'Lead_Bank_Name_x','Total_Investment_Exposure_x','Total_Exposure_x',
        'Assets_Classification_y','Assets_Classification']]

NPA_Class_v.columns=['Date', 'Template_Sheet_Name', 'Borrower_Customer_PAN','Borrower_Customer_Name', 'Assets_Classification',
       'Date_of_NPA_Classification', 'Bank_Name', 'Lead_Bank_Name','Total_Investment_Exposure','Total_Exposure','Previous_Month_Assets_Classification','TwoMonth_back_Assets_Classification']
NPA_Class_v.sort_values(by=['Borrower_Customer_PAN'], ascending=True)
NPA_Class_v


#InDefault & Out of Default
indefault = indefault[['Bank Name', 'Date', 'Borrower Name',
       'Borrower PAN','Date of Default','Funded Amount Outstanding as on Reporting Date','Total Amount Outstanding (Funded + Non-funded) as on Reporting Date']]
indefault.columns= ['Bank Name', 'Date', 'Borrower Name', 'Borrower PAN', 'Date of Default',
       'FAO','Total Amount Outstanding']
outdefault =outdefault[['Bank Name', 'Date','Borrower PAN','Date of Date of moving out of default']]
outdefault.columns = ['Bank Name', 'Date', 'Borrower PAN','Date of moving out of default']
matchall = indefault.merge(outdefault, how = 'outer', on = ['Bank Name', 'Borrower PAN'])
matchall.dropna(subset=['Date_x'], inplace=True)
matchall.loc[matchall['Date of Default']>matchall['Date of moving out of default'],'Date of moving out of default']="31-12-2030"
matchall1 = matchall.sort_values(['Bank Name','Borrower PAN','Date_x','Date of Default','Date of moving out of default'],ascending=[True,True,True,True,True])
matchall1 = matchall1.drop_duplicates(subset=['Bank Name', 'Borrower PAN','Date of Default'], keep='first')
matchall1 = matchall1.reset_index()
matchall1.loc[matchall1['Date of moving out of default']== "31-12-2030",['Date of moving out of default','Date_y']]=""
matchall1.shape

C:\Users\admin\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:2169: UserWarning: Parsing '31-12-2030' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  arr._validate_setitem_value(element)
C:\Users\admin\anaconda3\lib\site-packages\pandas\core\arrays\_mixins.py:249: UserWarning: Parsing '31-12-2030' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  value = self._validate_setitem_value(value)
C:\Users\admin\anaconda3\lib\site-packages\pandas\core\arraylike.py:40: UserWarning: Parsing '31-12-2030' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return self._cmp_method(other, operator.eq)


(22712, 10)

In [50]:
# Duplicate Data/ Errorneous Data
Erroneous_records2 = matchall1[matchall1.duplicated(['Bank Name','Borrower PAN','Date of moving out of default'],keep= False)]
Erroneous_records2 = Erroneous_records2[['Bank Name', 'Date_x', 'Borrower Name', 'Borrower PAN',
       'Date of Default', 'FAO', 'Total Amount Outstanding', 'Date_y','Date of moving out of default']]
Erroneous_records2.columns= ['Bank_Name', 'Default_Reporting_Date', 'Borrower_Name', 'Borrower_PAN',
       'Date_of_Default', 'FAO', 'Total Amount Outstanding', 'Out_of_Default_Reporting_Date','Date_Move_Out']
RDB_Duplicate_Data = Erroneous_records2

In [51]:
# Borrower remain in Default
Asset_Data = ['Standard','Non-CDR Standard Restructured','Standard Restructured under CDR']
data = current[current['Assets_Classification'].isin(Asset_Data)]
data = data[data['Special_Mention_Accounts'].isna()]
df2 = data
df1 = matchall1[matchall1['Date of moving out of default'].isna()]
df1.columns=['index','Bank_Name', 'Date_x', 'Borrower_Customer_Name', 'Borrower_Customer_PAN','Date of Default', 'FAO', 'Total Amount Outstanding', 'Date_y','Date of moving out of default']
merge = df2.merge(df1, how = 'inner', on = ['Bank_Name','Borrower_Customer_PAN'])
merge = merge[['Date','Bank_Name','Date_x','Template_Sheet_Name','Borrower_Customer_Name_x','Borrower_Customer_PAN','Total_Exposure','Date of Default','Date of moving out of default','Date_y','Special_Mention_Accounts']]
merge.columns = ['Date', 'Bank_Name','Default_Reporting_Date','Template_Sheet_Name', 'Borrower_Customer_Name','Borrower_Customer_PAN','Total_Exposure', 'Date_of_Default','Date_Move_Out','Out_of_Default_Reporting_Date','Special_Mention_Accounts']
merge.sort_values(by=['Date_of_Default'], ascending=True)
merge['Special_Mention_Accounts'].fillna(value = "N.A.",inplace=True)
RDB_SMA_NA = merge

In [52]:
# Borrower move out of Default

bmod= matchall1
bmod = bmod.sort_values(by=['Date of Default'],ascending=[False])
bmod = bmod.drop_duplicates(subset=['Bank Name', 'Borrower PAN'],keep='first')
bmod = bmod[~bmod['Date of moving out of default'].isna()]
dt1 = current.merge(data, how='outer', indicator=True).query('_merge == "left_only"').drop('_merge', 1)

dt1 = dt1[['Date','Template_Sheet_Name','Borrower_Customer_PAN','Borrower_Customer_Name','Date_of_NPA_Classification',
           'Amount_technically_prudentially','Special_Mention_Accounts', 'Funded_Amount_Outstanding_Total','Bank_Name','Total_Exposure']]
bmod.columns = ['index', 'Bank_Name', 'Date_x','Borrower_Customer_Name' ,'Borrower_Customer_PAN',
       'Date of Default', 'FAO', 'Total Amount Outstanding', 'Date_y','Date of moving out of default']

merged_file = pd.merge(dt1,bmod, how = 'inner', on = ['Bank_Name','Borrower_Customer_PAN'])


merged_file = pd.merge(dt1,bmod, how = 'inner', on = ['Bank_Name','Borrower_Customer_PAN'])
merged_file = merged_file[['Date', 'Borrower_Customer_PAN',
       'Borrower_Customer_Name_x', 'Special_Mention_Accounts','Funded_Amount_Outstanding_Total', 'Bank_Name',
        'Date_x', 'Date of Default', 'FAO','Total Amount Outstanding', 'Date_y', 'Date of moving out of default']]
merged_file.columns = ['Date', 'Borrower_Customer_PAN',
       'Borrower_Customer_Name', 'Special_Mention_Accounts','Funded_Amount_Outstanding_Total', 'Bank_Name',
        'Default_Reporting_Date', 'Date_of_Default', 'FAO','Total_Amount_Outstanding', 'Out_Of_Default_Reporting', 'Date_Move_Out']
RDB_SMA_0_1_2_Null = merged_file

C:\Users\admin\AppData\Local\Temp\ipykernel_11848\3678429729.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dt1 = current.merge(data, how='outer', indicator=True).query('_merge == "left_only"').drop('_merge', 1)


In [53]:
RDB_SMA_0_1_2_Null.shape

(206, 12)

In [54]:
current['Bank_Name'].unique()

array(['FEDERAL BANK LTD', 'STATE BANK OF INDIA', 'CANARA BANK',
       'UNION BANK OF INDIA', 'UCO BANK', 'INDIAN BANK', 'BANK OF INDIA',
       'YES BANK LTD.', 'CENTRAL BANK OF INDIA', 'PUNJAB NATIONAL BANK',
       'AXIS BANK LIMITED', 'BANK OF MAHARASHTRA',
       'SOUTH INDIAN BANK LTD', 'IDBI BANK LIMITED', 'RBL BANK LIMITED',
       'BANK OF BARODA', 'INDIAN OVERSEAS BANK', 'SBM BANK (INDIA) LTD.',
       'KOTAK MAHINDRA BANK LTD.', 'SMALL INDUSTRIES DEV. BANK OF INDIA',
       'HDFC BANK LTD.', 'DCB BANK LIMITED', 'ICICI BANK LIMITED',
       'CTBC BANK CO., LTD.', 'THE DHANALAKSHMI BANK LTD',
       'TAMILNAD MERCANTILE BANK LTD', 'KARNATAKA BANK LTD',
       'PUNJAB AND SIND BANK', 'IDFC FIRST BANK LIMITED',
       'AU SMALL FINANCE BANK LIMITED', 'DBS BANK INDIA LTD.',
       'STANDARD CHARTERED BANK', 'JAMMU & KASHMIR BANK LTD',
       'KARUR VYSYA BANK LTD', 'CSB BANK LIMITED',
       'JPMORGAN CHASE BANK NATIONAL ASSOCIATION',
       'CITY UNION BANK LIMITED', 'AMERICAN 

In [55]:
# Save to Excel
with pd.ExcelWriter("January_2023_Updated.xlsx") as writer:   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    
    CIN.to_excel(writer, sheet_name="CIN", index=False) 
    Cin_Individual.to_excel(writer, sheet_name="CIN_Individual", index=False) 
    Lei_Te.to_excel(writer, sheet_name="LEI_TE", index=False)
    LEI_Individual.to_excel(writer, sheet_name="LEI_Individual", index=False)
    Lead_Bank_exclude.to_excel(writer, sheet_name="Lead_Bank_exclude", index=False)
    Cin_Rbi.to_excel(writer, sheet_name="CIN_RBI", index=False)  
    SMA_Reporting.to_excel(writer, sheet_name="SMA_Reporting", index=False) 
    NPA_Class_i.to_excel(writer, sheet_name="NPA_Classification_i", index=False) 
    NPA_Class_ii.to_excel(writer, sheet_name="NPA_Classification_ii", index=False)
    NPA_Class_iii.to_excel(writer, sheet_name="NPA_Classification_iii", index=False)
    NPA_Class_iv.to_excel(writer, sheet_name="NPA_Classification_iv", index=False)
    NPA_Class_v.to_excel(writer, sheet_name="NPA_Classification_v", index=False)  
    CIN_PAN_Combo.to_excel(writer, sheet_name="CIN_PAN_Combo", index=False)
    LEI_PAN_Combo.to_excel(writer, sheet_name="LEI_PAN_Combo", index=False)
    External_Rating_Changes.to_excel(writer, sheet_name="External_Rating_Changes", index=False)
    Lead_Bank_Reporting.to_excel(writer, sheet_name="Lead_Bank_Reporting", index=False) 
#     Bank_level_SMA.to_excel(writer, sheet_name="Bank_level_SMA", index=False) 
    Industry_Classification.to_excel(writer, sheet_name="Industry_Classification", index=False) 
    Industry_code_change.to_excel(writer, sheet_name="Industry_code_change", index=False) 
    Under_reporting.to_excel(writer, sheet_name="Under_reporting", index=False) 
    miscellaneous.to_excel(writer, sheet_name="Miscellaneous", index=False)
    Borrower_Group.to_excel(writer, sheet_name="Borrower_Group", index=False) 
    RDB_Duplicate_Data.to_excel(writer, sheet_name="RDB_Duplicate_Data", index=False)
    RDB_SMA_NA.to_excel(writer, sheet_name="RDB_SMA_NA", index=False)
    RDB_SMA_0_1_2_Null.to_excel(writer, sheet_name="RDB_SMA_0_1_2_Null", index=False)
    
    writer.save()

C:\Users\admin\anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [67]:
# To Save the Output in Bank Wise.

mylist= [CIN,Cin_Individual,Cin_Rbi,CIN_PAN_Combo,Lei_Te,LEI_Individual,LEI_PAN_Combo,External_Rating_Changes,
Lead_Bank_Reporting,Lead_Bank_exclude,Industry_Classification,Industry_code_change,Borrower_Group,Under_reporting,miscellaneous,
SMA_Reporting,NPA_Class_i,NPA_Class_ii,NPA_Class_iii,NPA_Class_iv,NPA_Class_v,RDB_Duplicate_Data,RDB_SMA_NA,RDB_SMA_0_1_2_Null]

Sheet_name= ["CIN","Cin_Individual","Cin_Rbi","CIN_PAN_Combo","Lei_Te","LEI_Individual","LEI_PAN_Combo","External_Rating_Changes",
            "Lead_Bank_Reporting","Lead_Bank_exclude","Industry_Classification","Industry_code_change","Borrower_Group","Under_reporting",
             "miscellaneous","SMA_Reporting","NPA_Class_i","NPA_Class_ii","NPA_Class_iii","NPA_Class_iv","NPA_Class_v","RDB_Duplicate_Data",
             "RDB_SMA_NA","RDB_SMA_0_1_2_Null"]

all_bank_names = current['Bank_Name'].unique()
bank_name = all_bank_names

# writer = pd.ExcelWriter("output_file.xlsx", engine = 'openpyxl')  

for bank in bank_name:
    #for data in mylist:
        #for sheet_name in Sheet_name:
    writer = pd.ExcelWriter(f"{bank}.xlsx", engine = 'openpyxl')
    
    df1 = CIN[CIN["Bank_Name"] == bank]
    df2 = Cin_Individual[Cin_Individual["Bank_Name"] == bank]
    df3 = Cin_Rbi[Cin_Rbi["Bank_Name"] == bank]
    df4 = CIN_PAN_Combo[CIN_PAN_Combo["Bank_Name"] == bank]
    df5 = Lei_Te[Lei_Te["Bank_Name"] == bank]
    df6 = LEI_Individual[LEI_Individual["Bank_Name"] == bank]
    df7 = LEI_PAN_Combo[LEI_PAN_Combo["Bank_Name"] == bank]
    df8 = External_Rating_Changes[External_Rating_Changes["Bank_Name"] == bank]
    df9 = Lead_Bank_Reporting[Lead_Bank_Reporting["Bank_Name"] == bank]
    df10 = Lead_Bank_exclude[Lead_Bank_exclude["Bank_Name"] == bank]
    df11 = Industry_Classification[Industry_Classification["Bank_Name"] == bank]
    df12 = Industry_code_change[Industry_code_change["Bank_Name"] == bank]
    df13 = Borrower_Group[Borrower_Group["Bank_Name"] == bank]
    df14 = Under_reporting[Under_reporting["Bank_Name"] == bank]
    df15 = miscellaneous[miscellaneous["Bank_Name"] == bank]
#     df16 = Bank_level_SMA[Bank_level_SMA["Bank_Name"] == bank]
    df17 = SMA_Reporting[SMA_Reporting["Bank_Name"] == bank]
    df18 = NPA_Class_i[NPA_Class_i["Bank_Name"] == bank]
    df19 = NPA_Class_ii[NPA_Class_ii["Bank_Name"] == bank]
    df20 = NPA_Class_iii[NPA_Class_iii["Bank_Name"] == bank]
    df21 = NPA_Class_iv[NPA_Class_iv["Bank_Name"] == bank]
    df22 = NPA_Class_v[NPA_Class_v["Bank_Name"] == bank]
    df23 = RDB_Duplicate_Data[RDB_Duplicate_Data["Bank_Name"] == bank]
    df24 = RDB_SMA_NA[RDB_SMA_NA["Bank_Name"] == bank]
    df25 = RDB_SMA_0_1_2_Null[RDB_SMA_0_1_2_Null["Bank_Name"] == bank]
    
    number_of_row = df1.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:    
            df1.to_excel(writer,sheet_name = "CIN",index = False)
            writer.save()
        
#     if df1.iloc[1:,1:].empty == False:     
#         df1.to_excel(writer,sheet_name = "CIN",index = False)
#         writer.save()    
    number_of_row = df2.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:
            df2.to_excel(writer,sheet_name = "Cin_Individual",index = False)
            writer.save()   
        
    number_of_row = df3.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:
            df3.to_excel(writer,sheet_name = "Cin_Rbi",index = False)
            writer.save()    
        
    number_of_row = df4.shape[0]
    if number_of_row > 0: 
        if number_of_row > 0:
            df4.to_excel(writer,sheet_name = "CIN_PAN_Combo",index = False)
            writer.save()  
        
    number_of_row = df5.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:
            df5.to_excel(writer,sheet_name = "Lei_Te",index = False)
            writer.save()  
        
    number_of_row = df6.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:
            df6.to_excel(writer,sheet_name = "LEI_Individual",index = False)
            writer.save()  
            
    number_of_row = df7.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:
            df7.to_excel(writer,sheet_name = "LEI_PAN_Combo",index = False)
            writer.save()
        
    number_of_row = df8.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:
            df8.to_excel(writer,sheet_name = "External_Rating_Changes",index = False)
            writer.save()    
        
    number_of_row = df9.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:
            df9.to_excel(writer,sheet_name = "Lead_Bank_Reporting",index = False)
            writer.save()    
        
    number_of_row = df10.shape[0]
    if number_of_row > 0:   
        if number_of_row > 0:
            df10.to_excel(writer,sheet_name = "Lead_Bank_exclude",index = False)
            writer.save()    
        
    number_of_row = df11.shape[0]
    if number_of_row > 0:  
        if number_of_row > 0:
            df11.to_excel(writer,sheet_name = "Industry_Classification",index = False)
            writer.save()  
        
    number_of_row = df12.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:
            df12.to_excel(writer,sheet_name = "Industry_code_change",index = False)
            writer.save()
        
    number_of_row = df13.shape[0]
    if number_of_row > 0:   
        if number_of_row > 0:
            df13.to_excel(writer,sheet_name = "Borrower_Group",index = False)
            writer.save() 
        
    number_of_row = df14.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:
            df14.to_excel(writer,sheet_name = "Under_reporting",index = False)
            writer.save()  
        
    number_of_row = df15.shape[0]
    if number_of_row > 0:  
        if number_of_row > 0:
            df15.to_excel(writer,sheet_name = "miscellaneous",index = False)
            writer.save()  
        
#     if df16.iloc[1:,:].empty == False:   
#         df16.to_excel(writer,sheet_name = "Bank_level_SMA",index = False)
#         writer.save()  
    number_of_row = df17.shape[0]
    if number_of_row > 0:  
        if number_of_row > 0:
            df17.to_excel(writer,sheet_name = "SMA_Reporting",index = False)
            writer.save()  
        
    number_of_row = df18.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:
            df18.to_excel(writer,sheet_name = "NPA_Class_i",index = False)
            writer.save()  
        
    number_of_row = df19.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:
            df19.to_excel(writer,sheet_name = "NPA_Class_ii",index = False)
            writer.save()  
        
    number_of_row = df20.shape[0]
    if number_of_row > 0: 
        if number_of_row > 0:
            df20.to_excel(writer,sheet_name = "NPA_Class_iii",index = False)
            writer.save() 
        
    number_of_row = df21.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:
            df21.to_excel(writer,sheet_name = "NPA_Class_iv",index = False)
            writer.save() 
        
    number_of_row = df22.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:
            df22.to_excel(writer,sheet_name = "NPA_Class_v",index = False)
            writer.save() 
        
    number_of_row = df23.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:
            df23.to_excel(writer,sheet_name = "RDB_Duplicate_Data",index = False)
            writer.save() 
        
    number_of_row = df24.shape[0]
    if number_of_row > 0: 
        if number_of_row > 0:
            df24.to_excel(writer,sheet_name = "RDB_SMA_NA",index = False)
            writer.save()  
        
    number_of_row = df25.shape[0]
    if number_of_row > 0:
        if number_of_row > 0:
            df25.to_excel(writer,sheet_name = "RDB_SMA_0_1_2_Null",index = False)      
            writer.save()    
    

In [72]:
number_of_row = df1.shape[0]
    if number_of_row > 0:    
        df1.to_excel(writer,sheet_name = "CIN",index = False)      
        writer.save()   

,Bank Name,To,cc
0,BANK OF BARODA,CREDIT.BCC@BANKOFBARODA.COM,cgm.compliance@bankofbaroda.com;gm.compliance....
1,BANK OF INDIA,HeadOffice.CreditMonitoring@bankofindia.co.in,Abhijit.Bose@bankofindia.co.in;HeadOffice.Comp...
2,BANK OF MAHARASHTRA,ranjit.singh@mahabank.co.in,dgmcmpl@mahabank.co.in;ssmbom@rbi.org.in;rohit...
3,CANARA BANK,homiscrilc@canarabank.com,ajithjose@canarabank.com;hocompliancedept@cana...
4,CENTRAL BANK OF INDIA,agm1creditmon@centralbank.co.in,gmcompliance@centralbank.co.in;ssm.cbi@rbi.org...
...,...,...,...
89,NORTH EAST SMALL FINANCE BANK LIMITED,rajibkumar.dutta@nesfb.com,prem.prakash1@nesfb.com;compliance@nesfb.com;s...
90,SHIVALIK SMALL FINANCE BANK LIMITED,rbicommunications@shivalikbank.com,pkshukla@shivalikbank.com;;prasannaagrawal@rbi...
91,SURYODAY SMALL FINANCE BANK LIMITED,ravikumar.arunachalam@suryodaybank.com,shankar.surendran@suryodaybank.com;;harishg@rb...
92,UJJIVAN SMALL FINANCE BANK LIMITED,nitin.kadam@ujjivan.com,brajesh.cherian@ujjivan.com;ssmujjivan@rbi.org...


In [73]:
import os
os.getcwd()

'C:\\Users\\Admin\\CRILC_DQL'

In [74]:
# #data.set_index("Bank Name",inplace = True)
# class EmailsSender:
#     def __init__(self):
#         self.outlook = win32.Dispatch('outlook.application')

#     def send_email(self, to_email_address, cc_email_address,attachment_path,attachment_path_1):
        
#         mail = self.outlook.CreateItem(0)
#         mail.To = to_email_address
#         mail.CC = cc_email_address
#         mail.Subject = 'Report'
#         mail.Body = """Report is attached."""
        
#         try:
#             if attachment_path:
#                 mail.Attachments.Add(Source=attachment_path)

#             if attachment_path_1:
#                 mail.Attachments.Add(Source=attachment_path_1)
# #             mail.Send()  
#         except:
#             print("Bank is not present")
                

#     def send_emails(self, to_email_address,cc_email_address, attachment_path=None,attachment_path_1 = None):
#             self.send_email(to_email_address,cc_email_address, attachment_path,attachment_path_1)
            
            
# for _,i,To,CC in Bank_data.itertuples():
#     attachment_path = f"C:\\Users\\Admin\\CRILC_DQL\\{i}.xlsx"
#     attachment_path_1 = "C:\\Users\\Admin\\CRILC_DQL\\Errors Description - New v2 - March 08 2023.pdf"
#     email_sender = EmailsSender()
#     email_sender.send_emails(To, CC ,attachment_path,attachment_path_1)

Bank is not present
Bank is not present
Bank is not present
Bank is not present
Bank is not present
Bank is not present


In [ ]:
# def send_mail(data):
    
#     for i,To,CC in data.itertuples():
        
#         #specifying application
#         outlook = win32.Dispatch('outlook.application')

#         # creating the object
#         mail = outlook.CreateItem(0)
        
#         #mail to be send
#         if (type(To) == str or type(CC) == str):
#             if type(To) == str:
#                 mail.To = To
#             if type(CC) == str:
#                 mail.CC= CC
                    
#         mail.Subject = ""
        
#         mail.add
                
#         #mail.Body = ''
#         mail.HTMLBody = f"""
#         Madam/Dear Sir,<br><br>
        
#                 """
#         mail.Send()

In [ ]:
# for i,_,To,To1,CC in data.itertuples():
    
#     for j,NBFC_Name,_return,_ in pending_return.itertuples():
        
#         if i == j:
#             if type(_return) is not str:
#                 if math.isnan(_return):
#                     continue
#             else:
                
#                 #specifying application
#                 outlook = win32.Dispatch('outlook.application')

#                 # creating the object
#                 mail = outlook.CreateItem(0)

#                 #mail to be send
#                 if (type(To) == str or type(To1) == str or  type(CC) == str):
#                     print("send mail")
#                     if type(To) == str:
#                         mail.To = To
#                     if type(To1) == str:
#                         mail.To = To1
#                     if type(CC) == str:
#                         mail.CC =CC
#                 else:
#                     continue
                    
#                 mail.Subject = data["NBFC Name"][i]+" "+"Pending return submission"
#                 #mail.Body = ''
#                 mail.HTMLBody = f"""
#                 Madam/Dear Sir,<br><br>
#                 It is observed that Company is yet to submit following returns for the period {pending_return["Reporting Peiod"][i]}<br><br>
#                 {pending_return["Pending Returns"][i]}<br><br><br>
#                 You are requested to arrange to submit the return without further delay<br><br>
#                 सादर / Regards<br>
#                 आर एम भट्ट/R.M.Bhat<br>
#                 सहायक महाप्रबंधक/AGM<br>
#                 डेटा & सांखिकी ग्रुप / Data & Statistics Group<br>
#                 पर्यवेक्षण विभाग, कें. का./ Department of Supervision, CO<br>
#                 भारतीय रिज़र्व बैंक/ Reserve Bank of India<br>
#                 """
#                 mail.Send()
